In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn import metrics

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import normalize
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
composition = pd.read_csv('gdrive/My Drive/cofs/composition_features2.csv')
structure = pd.read_csv('gdrive/My Drive/cofs/structural_features.csv')
properties = pd.read_csv('gdrive/My Drive/cofs/properties.csv')

In [ ]:
properties = properties.drop("Unnamed: 0", axis = 1)
structure = structure.drop("Unnamed: 0", axis = 1)

In [ ]:
properties.to_csv('gdrive/My Drive/cofs/properties.csv')

In [ ]:
properties.head()

,absoluteMethaneUptakeHighP_mol_kg_,deliverableCapacity_vSTP_v_,heatDesorptionHighP_kJ_mol_,CO2Qst_kJ_mol_
0,23.076498,179.615836,15.97766,-16.109667
1,27.624233,127.838063,9.32583,-13.759106
2,29.742485,119.016669,8.85691,-14.338564
3,33.747905,117.909108,8.84098,-12.775519
4,29.632996,126.083029,9.20892,-15.599998


In [ ]:
composition.head()

,NumC_A,NumN_A,NumO_A,NumF_A,NumCl_A,NumBr_A,NumB_A,NumSi_A,NumS_A,NumC_B,NumN_B,NumO_B,NumF_B,NumCl_B,NumBr_B,NumB_B,NumSi_B,net,linkerA_con,linkerB_con,FractionCSP3_A,NumAliphaticCarbocycles_A,NumAliphaticHeterocycles_A,NumAliphaticRings_A,NumAromaticCarbocycles_A,NumAromaticHeterocycles_A,NumAromaticRings_A,NumHAcceptors_A,NumHDonors_A,NumHeteroatoms_A,NumRotatableBonds_A,NumSaturatedCarbocycles_A,NumSaturatedHeterocycles_A,NumSaturatedRings_A,RingCount_A,FractionCSP3_B,NumAliphaticCarbocycles_B,NumAliphaticHeterocycles_B,NumAliphaticRings_B,NumAromaticCarbocycles_B,NumAromaticHeterocycles_B,NumAromaticRings_B,NumHAcceptors_B,NumHDonors_B,NumHeteroatoms_B,NumRotatableBonds_B,NumSaturatedCarbocycles_B,NumSaturatedHeterocycles_B,NumSaturatedRings_B,RingCount_B
0,12,0,2,0,0,4,0,0,0,25,0,0,0,0,4,0,0,pts,CH2,NH,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.12,4,0,4,2,0,2,0,0,4,0,0,0,0,6
1,12,0,2,0,0,4,0,0,0,6,2,4,0,0,2,0,0,qtz,CH2,NH,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.00,0,0,0,1,0,1,4,0,8,2,0,0,0,1
2,12,0,2,0,0,4,0,0,0,6,2,4,0,0,2,0,0,und,CH2,NH,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.00,0,0,0,1,0,1,4,0,8,2,0,0,0,1
3,12,0,2,0,0,4,0,0,0,6,4,0,0,0,2,0,0,unc,CH2,NH,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.00,0,0,0,1,0,1,4,4,6,0,0,0,0,1
4,12,0,2,0,0,4,0,0,0,6,0,4,0,0,2,0,0,qtz,CH2,NH,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.00,0,0,0,1,0,1,4,4,6,0,0,0,0,1


In [ ]:
categorical = composition[["net", "linkerA_con", "linkerB_con"]]
composition = composition.drop(["net", "linkerA_con", "linkerB_con"], axis = 1)

ohe = OneHotEncoder()
encoded_columns = ohe.fit_transform(categorical)

df = pd.DataFrame(encoded_columns.toarray())

composition = pd.concat([composition, df], axis = 1)

In [ ]:
composition.reset_index(drop=True, inplace=True)
structure.reset_index(drop=True, inplace=True)
combined_features = pd.concat([composition, structure], axis = 1)

In [ ]:
combined_features.head()

,NumC_A,NumN_A,NumO_A,NumF_A,NumCl_A,NumBr_A,NumB_A,NumSi_A,NumS_A,NumC_B,NumN_B,NumO_B,NumF_B,NumCl_B,NumBr_B,NumB_B,NumSi_B,FractionCSP3_A,NumAliphaticCarbocycles_A,NumAliphaticHeterocycles_A,NumAliphaticRings_A,NumAromaticCarbocycles_A,NumAromaticHeterocycles_A,NumAromaticRings_A,NumHAcceptors_A,NumHDonors_A,NumHeteroatoms_A,NumRotatableBonds_A,NumSaturatedCarbocycles_A,NumSaturatedHeterocycles_A,NumSaturatedRings_A,RingCount_A,FractionCSP3_B,NumAliphaticCarbocycles_B,NumAliphaticHeterocycles_B,NumAliphaticRings_B,NumAromaticCarbocycles_B,NumAromaticHeterocycles_B,NumAromaticRings_B,NumHAcceptors_B,NumHDonors_B,NumHeteroatoms_B,NumRotatableBonds_B,NumSaturatedCarbocycles_B,NumSaturatedHeterocycles_B,NumSaturatedRings_B,RingCount_B,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,density_kg_m_3_,poreVolume_cm_3_g_,largestIncludedSphereDiameter_A_,largestFreeSphereDiameter_A_,largestIncludedSphereAlongFreeSpherePathDiameter_A_,voidFraction_widom_
0,12,0,2,0,0,4,0,0,0,25,0,0,0,0,4,0,0,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.12,4,0,4,2,0,2,0,0,4,0,0,0,0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,428.773315,1.56524,8.95787,7.53243,8.95787,0.771182
1,12,0,2,0,0,4,0,0,0,6,2,4,0,0,2,0,0,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.00,0,0,0,1,0,1,4,0,8,2,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
combined_features.to_csv("gdrive/My Drive/cofs/combined_features2.csv")

In [ ]:
pd.set_option('display.max_columns', None)
combined_features.head()

,NumC_A,NumN_A,NumO_A,NumF_A,NumCl_A,NumBr_A,NumB_A,NumSi_A,NumS_A,NumC_B,NumN_B,NumO_B,NumF_B,NumCl_B,NumBr_B,NumB_B,NumSi_B,FractionCSP3_A,NumAliphaticCarbocycles_A,NumAliphaticHeterocycles_A,NumAliphaticRings_A,NumAromaticCarbocycles_A,NumAromaticHeterocycles_A,NumAromaticRings_A,NumHAcceptors_A,NumHDonors_A,NumHeteroatoms_A,NumRotatableBonds_A,NumSaturatedCarbocycles_A,NumSaturatedHeterocycles_A,NumSaturatedRings_A,RingCount_A,FractionCSP3_B,NumAliphaticCarbocycles_B,NumAliphaticHeterocycles_B,NumAliphaticRings_B,NumAromaticCarbocycles_B,NumAromaticHeterocycles_B,NumAromaticRings_B,NumHAcceptors_B,NumHDonors_B,NumHeteroatoms_B,NumRotatableBonds_B,NumSaturatedCarbocycles_B,NumSaturatedHeterocycles_B,NumSaturatedRings_B,RingCount_B,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,density_kg_m_3_,poreVolume_cm_3_g_,largestIncludedSphereDiameter_A_,largestFreeSphereDiameter_A_,largestIncludedSphereAlongFreeSpherePathDiameter_A_,voidFraction_widom_
0,12,0,2,0,0,4,0,0,0,25,0,0,0,0,4,0,0,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.12,4,0,4,2,0,2,0,0,4,0,0,0,0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,428.773315,1.56524,8.95787,7.53243,8.95787,0.771182
1,12,0,2,0,0,4,0,0,0,6,2,4,0,0,2,0,0,0.0,0,0,0,2,0,2,2,2,6,1,0,0,0,2,0.00,0,0,0,1,0,1,4,0,8,2,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,